In [4]:
import pandas as pd 

```
select  c.table_name, c.column_name, c.data_type, c.*
from information_schema.columns c
where 1=1
and c.table_catalog = 'faculty-Cornell-CS'
and c.table_schema = 'main'
and c.table_name in (
	--'t_note'
	select table_name from information_schema.tables t
	where t.table_catalog = 'faculty-Cornell-CS'
	and t.table_schema = 'main'
)
order by c.table_catalog,c.table_schema,c.table_name, c.ordinal_position
```

In [5]:
df = pd.read_csv("columns_202304020735.csv")

In [7]:
df = df[["table_name","column_name"]]

In [12]:
df_list= df.groupby('table_name')['column_name'].apply(list)

In [15]:
TABLE_COLUMNS = df_list.to_dict()

In [16]:
TABLE_COLUMNS

{'t_faculty': ['name',
  'job_title',
  'phd_univ',
  'phd_year',
  'research_area',
  'research_concentration',
  'research_focus',
  'url',
  'img_url',
  'phone',
  'email',
  'cell_phone',
  'office_address',
  'department',
  'school'],
 't_note': ['title', 'note', 'url', 'tags', 'ts', 'id'],
 't_research_group': ['research_group', 'url']}

In [1]:
s = """
title
note
url
tags
ts
id
"""

In [2]:
cols = [i.strip() for i in s.split("\n") if i.strip()]

In [3]:
cols

['title', 'note', 'url', 'tags', 'ts', 'id']

In [17]:
'title'.capitalize()

'Title'